# Priority Queues & Heaps Week-plan

This notebook consolidates all theory, pseudocode, and Python implementations for priority queues and heaps.

## 1. Binary Heaps & Basic Priority-Queue Operations

**Heap Representation & Properties**
- A **binary heap** is a complete binary tree stored in an array `A[1…n]`.
- Parent/child relationship:
  - Parent of node `i` is at `⌊i/2⌋`.
  - Children of `i` are at `2i` (left) and `2i+1` (right), if ≤ `n`.
- **Max-heap property**: `A[parent(i)] ≥ A[i]` for all `i>1`.
- Height `h = ⌊log₂ n⌋`; number of nodes `n = 2^{h+1} – 1`.

### 1.1 MAX-HEAPIFY (O(log n))
Restores the max-heap property at node `i`, assuming its children are roots of valid heaps.
```text
MAX-HEAPIFY(A, i, n):
  l ← 2i; r ← 2i+1
  largest ← i
  if l ≤ n and A[l] > A[largest]:
    largest ← l
  if r ≤ n and A[r] > A[largest]:
    largest ← r
  if largest ≠ i:
    swap A[i], A[largest]
    MAX-HEAPIFY(A, largest, n)
```

In [ ]:
def max_heapify(A, i, heap_size):
    l, r = 2*i, 2*i + 1
    largest = i
    if l <= heap_size and A[l] > A[largest]:
        largest = l
    if r <= heap_size and A[r] > A[largest]:
        largest = r
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        max_heapify(A, largest, heap_size)

### 1.2 BUILD-MAX-HEAP (O(n))
Transforms an unordered array into a max-heap.
```text
BUILD-MAX-HEAP(A):
  heap_size ← n
  for i from ⌊n/2⌋ downto 1:
    MAX-HEAPIFY(A, i, heap_size)
```

In [ ]:
def build_max_heap(A):
    heap_size = len(A) - 1
    for i in range(heap_size//2, 0, -1):
        max_heapify(A, i, heap_size)
    return heap_size

### 1.3 HEAP-EXTRACT-MAX & HEAP-INCREASE-KEY (O(log n))
```text
HEAP-EXTRACT-MAX(A):
  if heap_size < 1: error
  max ← A[1]
  A[1] ← A[heap_size]
  heap_size ← heap_size−1
  MAX-HEAPIFY(A, 1, heap_size)
  return max

HEAP-INCREASE-KEY(A, i, key):
  if key < A[i]: error
  A[i] ← key
  while i>1 and A[parent(i)] < A[i]:
    swap A[i], A[parent(i)]
    i ← parent(i)
```

In [ ]:
def heap_extract_max(A, heap_size):
    if heap_size < 1:
        raise IndexError("Underflow")
    maximum = A[1]
    A[1] = A[heap_size]
    heap_size -= 1
    max_heapify(A, 1, heap_size)
    return maximum, heap_size

def heap_increase_key(A, i, key):
    if key < A[i]:
        raise ValueError("New key smaller than current")
    A[i] = key
    while i > 1 and A[i//2] < A[i]:
        A[i], A[i//2] = A[i//2], A[i]
        i //= 2

### 1.4 MAX-HEAP-INSERT (O(log n))
```text
MAX-HEAP-INSERT(A, key):
  heap_size ← heap_size+1
  A[heap_size] ← −∞
  HEAP-INCREASE-KEY(A, heap_size, key)
```

In [ ]:
def max_heap_insert(A, heap_size, key):
    heap_size += 1
    A[heap_size] = float('-inf')
    heap_increase_key(A, heap_size, key)
    return heap_size

## 2. Merging k Sorted Arrays in O(n log k)
Use a min-heap of size k to merge.
```text
MERGE-K-ARRAYS(lists[1…k]):
  create empty min-heap H
  for i from 1 to k:
    if lists[i] nonempty:
      H.insert( (lists[i][0], i, 0) )
  result ← []
  while H not empty:
    (v, i, j) ← H.extract_min()
    append v to result
    if j+1 < len(lists[i]):
      H.insert( (lists[i][j+1], i, j+1) )
  return result
```

In [ ]:
import heapq

def merge_k_sorted(lists):
    H = []
    for i, lst in enumerate(lists):
        if lst:
            heapq.heappush(H, (lst[0], i, 0))
    result = []
    while H:
        v, i, j = heapq.heappop(H)
        result.append(v)
        if j+1 < len(lists[i]):
            heapq.heappush(H, (lists[i][j+1], i, j+1))
    return result

## 3. Extended Priority-Queue Operations
**DELETE(i)**: 
```text
DELETE(A, i):
  HEAP-INCREASE-KEY(A, i, +∞)
  HEAP-EXTRACT-MAX(A)
```
**FUSION(i, j)**: 
```text
FUSION(A, i, j):
  xi ← extract max at i
  xj ← extract max at j
  insert(xi + xj)
```
**THRESHOLD(k)** in O(m):
```text
THRESHOLD(A, heap_size, k):
  result ← []
  queue ← [1]
  while queue not empty:
    i ← dequeue(queue)
    if i>heap_size or A[i] < k: continue
    append A[i] to result
    enqueue(queue, 2i); enqueue(queue, 2i+1)
  return result
```

In [ ]:
from collections import deque

def threshold(A, heap_size, k):
    res = []
    Q = deque([1])
    while Q:
        i = Q.popleft()
        if i > heap_size or A[i] < k:
            continue
        res.append(A[i])
        Q.append(2*i)
        Q.append(2*i+1)
    return res

## 4. Satellite Data
Store satellite data with each key, e.g., tuples `(key, data)`. Heap operations compare on `key`. Use parallel arrays or tuple swapping as needed.

## 5. Heap-Size & Height Relations
- Nodes `n = 2^{h+1} - 1` for height `h`.
- Height `h = ⌊log₂ n⌋`.
- Build-heap runs in `O(n)` via level-by-level heapify summation.

## 6. Task Delegation via Max-Heap
Support `NEWTASK(id, diff)` and `REQUESTTASK()` by maintaining a max-heap of `(diff, id)`.

In [ ]:
import heapq

class TaskDelegator:
    def __init__(self):
        self.H = []  # min-heap on -diff
    def new_task(self, tid, diff):
        heapq.heappush(self.H, (-diff, tid))
    def request_task(self):
        if not self.H:
            return None
        diff, tid = heapq.heappop(self.H)
        return tid

## 7. Range-Sum Queries & Point Updates
### 7.1 Fenwick Tree (BIT)
O(log n) updates & prefix-sum queries.

In [ ]:
class Fenwick:
    def __init__(self, A):
        self.n = len(A)
        self.B = [0]*(self.n+1)
        for i, v in enumerate(A, 1):
            self._update(i, v)
    def _update(self, i, delta):
        while i <= self.n:
            self.B[i] += delta
            i += i & -i
    def change(self, i, x):
        old = self.query(i, i)
        self._update(i+1, x-old)
    def prefix(self, i):
        s = 0
        i += 1
        while i > 0:
            s += self.B[i]
            i -= i & -i
        return s
    def query(self, i, j):
        return self.prefix(j) - (self.prefix(i-1) if i else 0)

### 7.2 Segment Tree
O(log n) updates & range-sum queries.

In [ ]:
class SegTree:
    def __init__(self, A):
        n = len(A)
        size = 1
        while size < n:
            size *= 2
        self.N = size
        self.T = [0]*(2*size)
        for i, v in enumerate(A):
            self.T[size+i] = v
        for i in range(size-1, 0, -1):
            self.T[i] = self.T[2*i] + self.T[2*i+1]
    def change(self, i, x):
        p = self.N + i
        self.T[p] = x
        while p > 1:
            p //= 2
            self.T[p] = self.T[2*p] + self.T[2*p+1]
    def sum(self, l, r):
        res = 0
        l += self.N; r += self.N
        while l <= r:
            if l & 1:
                res += self.T[l]; l += 1
            if not r & 1:
                res += self.T[r]; r -= 1
            l //= 2; r //= 2
        return res